In [1]:
%load_ext autoreload
%autoreload 2

import xarray as xr
import os
import pandas as pd
import numpy as np
import os
import netCDF4 as nc

In [ ]:
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=10, threads_per_worker=5, memory_limit='6GB')

# aggregate 500m HMASR to 10km resolution
def agg_HMASR(year, path, out_path):
    """aggregate the HMASR data to 10km resolution"""
    # parameters
    path = path+str(year)+'/'
    out_path = out_path+str(year)+'/'
    ds_name_list = ['FORCING_POST', 'SWE_SCA_POST', 'SD_POST']

    for ds_name in ds_name_list:
        # list all files
        list_files = [f for f in os.listdir(path) if ds_name in f] 

        ds_10km_mask = [] # apply non-seasonal snow mask
        for file in list_files:
            ds = delayed(xr.open_dataset)(path+file)
            mask = delayed(xr.open_dataset)(path+file.replace(ds_name, 'MASK'))
            if ds_name in ['FORCING_POST']:
                ds_10km_mask.append(ds.where(mask.Non_seasonal_snow_mask == 0).coarsen(Latitude=25, Longitude=25).mean())
            else:
                # Select only the mean (Stats=0)
                ds_10km_mask.append(ds.isel(Stats=0).where(mask.Non_seasonal_snow_mask == 0).coarsen(Latitude=25, Longitude=25).mean())
        ds_10km_mask = compute(ds_10km_mask)
        xr.save_mfdataset(ds_10km_mask[0], [out_path+file.replace(ds_name, ds_name+'_MASK') for file in list_files])

for i in range(2002, 2003):
    agg_HMASR(i, '/tera04/lilu/HMASR/', '/tera06/lilu/fSCA/data/')

In [ ]:
# combine the 10km resolution data to a single file
def combine_mfdata(year, path, out_path):
    ds_name_list = ['FORCING_POST', 'SWE_SCA_POST', 'SD_POST']

    for ds_name in ds_name_list:
        print(ds_name)
        ds = xr.open_mfdataset(path+str(year)+'/*'+str(year)+'*'+ds_name+'_MASK.nc')
        ds = ds.assign_coords(Day=pd.date_range(start='{year}-10-01'.format(year=year), periods=ds.Day.size, freq='D'))
        ds = ds.rename({'Longitude': 'lon', 'Latitude': 'lat', 'Day': 'time'}).transpose("time", "lat", "lon")
        if ds_name in ['FORCING_POST']:
            ds = xr.where(ds.Ta_Post>30, ds, np.nan)
            ds = ds.reindex(lat=list(reversed(ds.lat)))
        ds.to_netcdf(out_path+'HMA_SR_D_v01_10km_'+str(year)+'_'+ds_name+'_MASK.nc')

for i in range(2002, 2003):
    combine_mfdata(i, '/tera06/lilu/fSCA/data/', '/tera06/lilu/fSCA/data/')

In [ ]:
# generate 90m for the MERITHydro dataset
lat = np.zeros((6000*4,))*np.nan
for i, nlat in enumerate(range(40, 24, -5)):
    filename = os.path.join('/tera02/zhangsp/tera07/MERITHydro/n{lat:02}e095.nc'.format(lat=nlat))
    f = xr.open_dataset(filename)
    lat[6000*i:6000*(i+1)] = np.array(f['latitude'][:])

lon = np.zeros((6000*9,))*np.nan
for j, nlon in enumerate(range(60, 105, 5)):
    filename = os.path.join('/tera02/zhangsp/tera07/MERITHydro/n25e{lon:03}.nc'.format(lon=nlon))
    f = xr.open_dataset(filename)
    lon[6000*j:6000*(j+1)] = np.array(f['longitude'][:])

# 25-45N, 60-105E
elv = np.zeros((6000*9, 6000*4))*np.nan
for i, nlat in enumerate(range(40, 24, -5)):
    for j, nlon in enumerate(range(60, 105, 5)):
        print(nlat, nlon)
        filename = os.path.join('/tera02/zhangsp/tera07/MERITHydro/n{lat:02}e{lon:03}.nc'.format(lat=nlat, lon=nlon))
        f = xr.open_dataset(filename)
        elv[6000*j:6000*(j+1), 6000*i:6000*(i+1)] = np.array(f['elv'][:])

# save
f = nc.Dataset('MERITHydro_elv_90m_TP.nc', 'w', format='NETCDF4')
f.createDimension('longitude', size=lon.shape[0])
f.createDimension('latitude', size=lat.shape[0])

lon0 = f.createVariable('longitude', 'f4', dimensions='longitude')
lon0.units = "degrees_east"
lon0.long_name = "longitude"
lon0.standard_name = "longitude"
lon0.axis = "X"

lat0 = f.createVariable('latitude', 'f4', dimensions='latitude')
lat0.units = "degrees_north"
lat0.axis = "Y"
lat0.long_name = "latitude"
lat0.standard_name = "latitude"

data = f.createVariable('elv', 'f4', dimensions=('longitude','latitude'))
data.units = "m"
lon0[:], lat0[:], data[:] = lon, lat, elv
f.close()

In [ ]:
# calculate the subgrid standard deviation for the MERITHydro dataset
f = xr.open_dataset('MERITHydro_elv_90m_TP.nc')
elv_90m = np.array(f['elv'][:])
lat_90m, lon_90m = np.array(f['latitude'][:]), np.array(f['longitude'][:])
f = xr.open_dataset('MERITHydro_elv_0p1_TP.nc')
lat_0p1, lon_0p1 = np.array(f['lat'][:]), np.array(f['lon'][:])

# calculate the standard deviation
topo_std = np.zeros((len(lon_0p1), len(lat_0p1)))*np.nan
for i in range(len(lat_0p1)):
    for j in range(len(lon_0p1)):
        print(i, j)
        lat0, lon0 = lat_0p1[i], lon_0p1[j]

        # identify coordinates of select 500m grid
        if i == 0:
            lat1 = lat_0p1[i+1]
            dlat = np.abs((lat1-lat0)/2)
            blat, ulat = lat0-dlat, lat0+dlat
        elif i == len(lat_0p1)-1:
            lat1 = lat_0p1[i-1]
            dlat = np.abs((lat1-lat0)/2)
            blat, ulat = lat0-dlat, lat0+dlat
        else:
            lat1, lat2 = lat_0p1[i-1], lat_0p1[i+1]
            dlat1, dlat2 = np.abs((lat1-lat0)/2), np.abs((lat2-lat0)/2)
            blat, ulat = lat0-dlat1, lat0+dlat2

        if j == 0:
            lon1 = lon_0p1[j+1]
            dlon = np.abs((lon1-lon0)/2)
            llon, rlon = lon0-dlon, lon0+dlon
        elif j == len(lon_0p1)-1:
            lon1 = lon_0p1[j-1]
            dlon = np.abs((lon1-lon0)/2)
            llon, rlon = lon0-dlon, lon0+dlon
        else:  
            lon1, lon2 = lon_0p1[j-1], lon_0p1[j+1]
            dlon1, dlon2 = np.abs((lon1-lon0)/2), np.abs((lon2-lon0)/2)
            llon, rlon = lon0-dlon1, lon0+dlon2

        # find 90m grids locate in this grid
        idx_lat = np.where((lat_90m>=blat) & (lat_90m<=ulat))[0]
        idx_lon = np.where((lon_90m>=llon) & (lon_90m<=rlon))[0]
        tmp_elv = elv_90m[idx_lon,:][:,idx_lat]
        print(tmp_elv.shape)
        # calculate std
        topo_std[j, i] = np.nanstd(tmp_elv)

# save
f = nc.Dataset('MERITHydro_std_0p1_TP.nc', 'w', format='NETCDF4')
f.createDimension('lon', size=lon_0p1.shape[0])
f.createDimension('lat', size=lat_0p1.shape[0])

lon0 = f.createVariable('lon', 'f4', dimensions='lon')
lon0.units = "degrees_east"
lon0.long_name = "longitude"
lon0.standard_name = "longitude"
lon0.axis = "X"

lat0 = f.createVariable('lat', 'f4', dimensions='lat')
lat0.units = "degrees_north"
lat0.axis = "Y"
lat0.long_name = "latitude"
lat0.standard_name = "latitude"

data = f.createVariable('elv_std', 'f4', dimensions=('lon','lat'))
data.units = "m"
lon0[:], lat0[:], data[:] = lon_0p1, lat_0p1, topo_std
f.close()

In [16]:
# percentage of permanent snow in each 0.1 grid
ds = xr.open_mfdataset("/tera04/lilu/HMASR/1999/HMA_SR_D_v01_*_agg_16_WY1999_00_MASK.nc")
permanent_snow_count = ds.coarsen(Latitude=25, Longitude=25).sum().load()
permanent_snow_count = permanent_snow_count.rename({'Longitude': 'lon', 'Latitude': 'lat', 'Water_Year': 'year'}).transpose("year","lat", "lon")
mask_psnow = (permanent_snow_count/(25*25)*100)
mask_psnow.Non_seasonal_snow_mask.rename('P_psnow').to_netcdf('ratio_psnow.nc')

In [ ]:
def slice_lai_day(ds):
    out = []
    for i in range(2000, 2014):
        out.append(ds.sel(time=slice('{year}-01-01'.format(year=i), '{year}-12-31'.format(year=i), 8)).values)
    out = np.concatenate(out, axis=0)
    return out

ds = xr.open_dataset('SD.nc')
sd_train = slice_lai_day(ds.SD_Post)
print(out.shape)

# Making training data

## 1. Using all avaliable raw data (aware of dimensonless)

In [2]:
# make train data
# define train and test period
train_period = slice('1999-10-01', '2013-09-30') # ~80%
test_period = slice('2013-10-01', '2017-09-30') # ~20%

# read snow depth
ds = xr.open_dataset('SD.nc')
sd_train = ds.SD_Post.sel(time=train_period).values
sd_test = ds.SD_Post.sel(time=test_period).values
nt_train, nt_test = sd_train.shape[0], sd_test.shape[0]
lat, lon = ds.lat, ds.lon

# read snow water equivalent and snow cover
ds = xr.open_dataset('SWE_SCA.nc')
swe_train = ds.SWE_Post.sel(time=train_period).values
swe_test = ds.SWE_Post.sel(time=test_period).values
sca_train = ds.SCA_Post.sel(time=train_period).values
sca_test = ds.SCA_Post.sel(time=test_period).values
rho_train = swe_train/sd_train
rho_test = swe_test/sd_test
rho_train[np.where((rho_train>1000) | (rho_train<0))] = np.nan
rho_test[np.where((rho_test>1000) | (rho_test<0))] = np.nan

# read topo-related vars
ds = xr.open_dataset('MERITHydro/MERITHydro_0p1_TP.nc')
elv_std = ds.elv_std.values[np.newaxis]   
elv = ds.elv.values[np.newaxis]
slp = ds.slp.values[np.newaxis]
asp = ds.asp.values[np.newaxis]

# read forcing
ds = xr.open_dataset('FORCING.nc')
ta_train = ds.Ta_Post.sel(time=train_period).values
rs_train = ds.Rs_Post.sel(time=train_period).values
rl_train = ds.Rl_Post.sel(time=train_period).values
ps_train = ds.Ps_Post.sel(time=train_period).values
ppt_train = ds.PPT_Post.sel(time=train_period).values
q_train = ds.q_Post.sel(time=train_period).values

ta_test = ds.Ta_Post.sel(time=test_period).values
rs_test = ds.Rs_Post.sel(time=test_period).values
rl_test = ds.Rl_Post.sel(time=test_period).values
ps_test = ds.Ps_Post.sel(time=test_period).values
ppt_test = ds.PPT_Post.sel(time=test_period).values
q_test = ds.q_Post.sel(time=test_period).values

x_train = np.stack([sd_train, swe_train, ta_train, rs_train, rl_train, ps_train, ppt_train, q_train, \
    np.tile(elv_std, (nt_train,1,1)), np.tile(np.sin(slp), (nt_train,1,1)), np.tile(np.cos(asp), (nt_train,1,1))], axis=-1).reshape(-1, 11)
y_train = sca_train.reshape(-1,1)*100

x_test = np.stack([sd_test, swe_test, ta_test, rs_test, rl_test, ps_test, ppt_test, q_test, \
    np.tile(elv_std, (nt_test,1,1)), np.tile(np.sin(slp), (nt_test,1,1)), np.tile(np.cos(asp), (nt_test,1,1))], axis=-1).reshape(-1, 11)
y_test = sca_test.reshape(-1,1)*100

y_train = np.delete(y_train, np.where(np.isnan(x_train))[0], axis=0)
x_train = np.delete(x_train, np.where(np.isnan(x_train))[0], axis=0)
x_train = np.delete(x_train, np.where(np.isnan(y_train))[0], axis=0)
y_train = np.delete(y_train, np.where(np.isnan(y_train))[0], axis=0)
y_train = np.delete(y_train, np.where(np.isinf(x_train))[0], axis=0)
x_train = np.delete(x_train, np.where(np.isinf(x_train))[0], axis=0)
x_train = np.delete(x_train, np.where(np.isinf(y_train))[0], axis=0)
y_train = np.delete(y_train, np.where(np.isinf(y_train))[0], axis=0)

y_test = np.delete(y_test, np.where(np.isnan(x_test))[0], axis=0)
x_test = np.delete(x_test, np.where(np.isnan(x_test))[0], axis=0)
x_test = np.delete(x_test, np.where(np.isnan(y_test))[0], axis=0)
y_test = np.delete(y_test, np.where(np.isnan(y_test))[0], axis=0)
y_test = np.delete(y_test, np.where(np.isinf(x_test))[0], axis=0)
x_test = np.delete(x_test, np.where(np.isinf(x_test))[0], axis=0)
x_test = np.delete(x_test, np.where(np.isinf(y_test))[0], axis=0)
y_test = np.delete(y_test, np.where(np.isinf(y_test))[0], axis=0)

np.save('x_train_a.npy', x_train)
np.save('y_train_a.npy', y_train)
np.save('x_test_a.npy', x_test)
np.save('y_test_a.npy', y_test)

/tmp/ipykernel_4132162/2111100786.py:19: RuntimeWarning: divide by zero encountered in divide
  rho_train = swe_train/sd_train
/tmp/ipykernel_4132162/2111100786.py:19: RuntimeWarning: invalid value encountered in divide
  rho_train = swe_train/sd_train
/tmp/ipykernel_4132162/2111100786.py:20: RuntimeWarning: divide by zero encountered in divide
  rho_test = swe_test/sd_test
/tmp/ipykernel_4132162/2111100786.py:20: RuntimeWarning: invalid value encountered in divide
  rho_test = swe_test/sd_test


## 2. Using preprocesses dimensionless data

In [3]:
# make train data using designed factors
# define train and test period
train_period = slice('1999-10-01', '2013-09-30') # ~80%
test_period = slice('2013-10-01', '2017-09-30') # ~20%

# read snow depth
ds = xr.open_dataset('SD.nc')
sd_train = ds.SD_Post.sel(time=train_period).values
sd_test = ds.SD_Post.sel(time=test_period).values
nt_train, nt_test = sd_train.shape[0], sd_test.shape[0]
lat, lon = ds.lat, ds.lon

# read snow water equivalent and snow cover
ds = xr.open_dataset('SWE_SCA.nc')
swe_train = ds.SWE_Post.sel(time=train_period).values
swe_test = ds.SWE_Post.sel(time=test_period).values
sca_train = ds.SCA_Post.sel(time=train_period).values
sca_test = ds.SCA_Post.sel(time=test_period).values
swe, sca = ds.SWE_Post.values, ds.SCA_Post.values
swe_max = np.nanmax(swe)

# read topo-related vars
ds = xr.open_dataset('MERITHydro/MERITHydro_0p1_TP.nc')
elv_std = ds.elv_std.values[np.newaxis]   

# read forcing
ds = xr.open_dataset('FORCING.nc')
ta_train = ds.Ta_Post.sel(time=train_period).values
q_train = ds.q_Post.sel(time=train_period).values
ta_test = ds.Ta_Post.sel(time=test_period).values
q_test = ds.q_Post.sel(time=test_period).values

# calculate own designed factors
rho_train = swe_train/sd_train
rho_test = swe_test/sd_test
rho_train[np.where((rho_train>1000) | (rho_train<0))] = np.nan
rho_test[np.where((rho_test>1000) | (rho_test<0))] = np.nan

x_train = np.stack([sd_train/0.01, 1/rho_train, swe_train/swe_max, ta_train/273.15, q_train, \
    200/np.tile(elv_std, (nt_train,1,1))], axis=-1).reshape(-1, 6)
y_train = sca_train.reshape(-1,1)*100

x_test = np.stack([sd_test/0.01, 1/rho_test, swe_test/swe_max, ta_test/273.15, q_test, \
    200/np.tile(elv_std, (nt_test,1,1))], axis=-1).reshape(-1, 6)
y_test = sca_test.reshape(-1,1)*100

y_train = np.delete(y_train, np.where(np.isnan(x_train))[0], axis=0)
x_train = np.delete(x_train, np.where(np.isnan(x_train))[0], axis=0)
x_train = np.delete(x_train, np.where(np.isnan(y_train))[0], axis=0)
y_train = np.delete(y_train, np.where(np.isnan(y_train))[0], axis=0)
y_train = np.delete(y_train, np.where(np.isinf(x_train))[0], axis=0)
x_train = np.delete(x_train, np.where(np.isinf(x_train))[0], axis=0)
x_train = np.delete(x_train, np.where(np.isinf(y_train))[0], axis=0)
y_train = np.delete(y_train, np.where(np.isinf(y_train))[0], axis=0)

y_test = np.delete(y_test, np.where(np.isnan(x_test))[0], axis=0)
x_test = np.delete(x_test, np.where(np.isnan(x_test))[0], axis=0)
x_test = np.delete(x_test, np.where(np.isnan(y_test))[0], axis=0)
y_test = np.delete(y_test, np.where(np.isnan(y_test))[0], axis=0)
y_test = np.delete(y_test, np.where(np.isinf(x_test))[0], axis=0)
x_test = np.delete(x_test, np.where(np.isinf(x_test))[0], axis=0)
x_test = np.delete(x_test, np.where(np.isinf(y_test))[0], axis=0)
y_test = np.delete(y_test, np.where(np.isinf(y_test))[0], axis=0)

np.save('x_train_d.npy', x_train)
np.save('y_train_d.npy', y_train)
np.save('x_test_d.npy', x_test)
np.save('y_test_d.npy', y_test)

/tmp/ipykernel_4132162/3331777666.py:34: RuntimeWarning: divide by zero encountered in divide
  rho_train = swe_train/sd_train
/tmp/ipykernel_4132162/3331777666.py:34: RuntimeWarning: invalid value encountered in divide
  rho_train = swe_train/sd_train
/tmp/ipykernel_4132162/3331777666.py:35: RuntimeWarning: divide by zero encountered in divide
  rho_test = swe_test/sd_test
/tmp/ipykernel_4132162/3331777666.py:35: RuntimeWarning: invalid value encountered in divide
  rho_test = swe_test/sd_test
/tmp/ipykernel_4132162/3331777666.py:39: RuntimeWarning: divide by zero encountered in divide
  x_train = np.stack([sd_train/0.01, 1/rho_train, swe_train/swe_max, ta_train/273.15, q_train, \
/tmp/ipykernel_4132162/3331777666.py:40: RuntimeWarning: divide by zero encountered in divide
  200/np.tile(elv_std, (nt_train,1,1))], axis=-1).reshape(-1, 6)
/tmp/ipykernel_4132162/3331777666.py:43: RuntimeWarning: divide by zero encountered in divide
  x_test = np.stack([sd_test/0.01, 1/rho_test, swe_test/